In [ ]:
!pip install qiskit

     |████████████████████████████████| 6.5 MB 12.3 MB/s 
     |████████████████████████████████| 18.0 MB 86 kB/s 
     |████████████████████████████████| 238 kB 65.7 MB/s 
     |████████████████████████████████| 200 kB 58.4 MB/s 
     |████████████████████████████████| 53 kB 3.0 MB/s 
     |████████████████████████████████| 1.6 MB 52.2 MB/s 
     |████████████████████████████████| 37.5 MB 185 kB/s 
     |████████████████████████████████| 943 kB 56.4 MB/s 
     |████████████████████████████████| 49 kB 9.1 MB/s 
     |████████████████████████████████| 49 kB 6.5 MB/s 
     |████████████████████████████████| 38.1 MB 1.3 MB/s 
     |████████████████████████████████| 3.6 MB 65.5 MB/s 
     |████████████████████████████████| 113 kB 59.6 MB/s 
  Created wheel for qiskit: filename=qiskit-0.34.2-py3-none-any.whl size=11805 sha256=de5aa732589ab02f89f08653621b3d4a57c3162a5eec63a5acca0f8780836e55
  Stored in directory: /root/.cache/pip/wheels/62/77/65/cda6eedfdd2a525bd3f479a4386930ae3088a1eb01f8c9

In [ ]:
from qiskit import ClassicalRegister, QuantumRegister, QuantumCircuit 
from qiskit import execute
from qiskit import BasicAer 
q = QuantumRegister(4) 
c = ClassicalRegister(4) 
backend = BasicAer.get_backend('qasm_simulator')

In [ ]:
# Set train test split
training_set = [[0, 1], [0.78861006, 0.61489363]] 
labels = [0, 1] 
test_set = [[-0.549, 0.836], [0.053 , 0.999]]

In [ ]:
# Angle Definition
test_angles = [4.30417579487669/2, 3.0357101997648965/2] 
training_angle = 1.3245021469658966/4

In [6]:
# Define the state preperation function
def prepare_state(q, c, angles):
 ancilla_qubit = q[0]
 index_qubit = q[1]
 data_qubit = q[2]
 class_qubit = q[3]
 circuit = QuantumCircuit(q, c)
 # Ancilla and index qubits into uniform superposition 
 circuit.h(ancilla_qubit)
 circuit.h(index_qubit)
 # Prepare the test vector
 circuit.cx(ancilla_qubit, data_qubit)
 circuit.u3(-angles[0], 0, 0, data_qubit)
 circuit.cx(ancilla_qubit, data_qubit)
 circuit.u3(angles[0], 0, 0, data_qubit)
# Flip the ancilla qubit > this moves the input 
# vector to the |0> state of the ancilla
 circuit.x(ancilla_qubit)
 circuit.barrier()
# Prepare the first training vector 
# [0,1] -> class 0 
# We can prepare this with a Toffoli
 circuit.ccx(ancilla_qubit, index_qubit, data_qubit) 
# Flip index qubit -> moves the first training vector to the 
# |0> state of the index qubit
 circuit.x(index_qubit)
 circuit.barrier() 
# Prepare the second training vector 
# [0.78861, 0.61489] -> class 1
 circuit.ccx(ancilla_qubit, index_qubit, data_qubit)
 circuit.cx(index_qubit, data_qubit)
 circuit.u3(angles[1], 0, 0, data_qubit)
 circuit.cx(index_qubit, data_qubit)
 circuit.u3(-angles[1], 0, 0, data_qubit)
 circuit.ccx(ancilla_qubit, index_qubit, data_qubit)
 circuit.cx(index_qubit, data_qubit)
 circuit.cx(index_qubit, data_qubit)
 circuit.u3(angles[1], 0, 0, data_qubit)
 circuit.barrier()
 # Flip the class label for training vector
 #2
 circuit.cx(index_qubit, class_qubit)
 circuit.barrier()
 return circuit

Next we generate circuit for preparing state with the first test instance.

In [7]:

# Circuit Generation
from qiskit.tools.visualization import circuit_drawer 
angles = [test_angles[0], training_angle] 
state_preparation_0 = prepare_state(q, c, angles) 
circuit_drawer(state_preparation_0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: The QuantumCircuit.u3 method is deprecated as of 0.16.0. It will be removed no earlier than 3 months after the release date. You should use QuantumCircuit.u instead, which acts identically. Alternatively, you can decompose u3 in terms of QuantumCircuit.p and QuantumCircuit.sx: u3(ϴ,φ,λ) = p(φ+π) sx p(ϴ+π) sx p(λ) (2 pulses on hardware).
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: The QuantumCircuit.u3 method is deprecated as of 0.16.0. It will be removed no earlier than 3 months after the release date. You should use QuantumCircuit.u instead, which acts identically. Alternatively, you can decompose u3 in terms of QuantumCircuit.p and QuantumCircuit.sx: u3(ϴ,φ,λ) = p(φ+π) sx p(ϴ+π) sx p(λ) (2 pulses on hardware).
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: DeprecationWarning: The Quan

┌───┐                                   ┌───┐        ░            ░      »
q0_0: ┤ H ├──■───────────────────────■────────┤ X ├────────░───■────────░───■──»
      ├───┤  │                       │        └───┘        ░   │  ┌───┐ ░   │  »
q0_1: ┤ H ├──┼───────────────────────┼─────────────────────░───■──┤ X ├─░───■──»
      └───┘┌─┴─┐┌─────────────────┐┌─┴─┐┌────────────────┐ ░ ┌─┴─┐└───┘ ░ ┌─┴─┐»
q0_2: ─────┤ X ├┤ U3(-2.1521,0,0) ├┤ X ├┤ U3(2.1521,0,0) ├─░─┤ X ├──────░─┤ X ├»
           └───┘└─────────────────┘└───┘└────────────────┘ ░ └───┘      ░ └───┘»
q0_3: ─────────────────────────────────────────────────────░────────────░──────»
                                                           ░            ░      »
c0: 4/═════════════════════════════════════════════════════════════════════════»
                                                                               »
«                                                                      »
«q0_0: ───────────────────────────────────────────────────■────────────»
«                                                         │            »
«q0_1: ──■───────────────────────■────────────────────────■────■────■──»
«      ┌─┴─┐┌─────────────────┐┌─┴─┐┌──────────────────┐┌─┴─┐┌─┴─┐┌─┴─┐»
«q0_2: ┤ X ├┤ U3(0.33113,0,0) ├┤ X ├┤ U3(-0.33113,0,0) ├┤ X ├┤ X ├┤ X ├»
«      └───┘└─────────────────┘└───┘└──────────────────┘└───┘└───┘└───┘»
«q0_3: ────────────────────────────────────────────────────────────────»
«                                                                      »
«c0: 4/════════════════════════════════════════════════════════════════»
«                                                                      »
«                          ░       ░ 
«q0_0: ────────────────────░───────░─
«                          ░       ░ 
«q0_1: ────────────────────░───■───░─
«      ┌─────────────────┐ ░   │   ░ 
«q0_2: ┤ U3(0.33113,0,0) ├─░───┼───░─
«      └─────────────────┘ ░ ┌─┴─┐ ░ 
«q0_3: ────────────────────░─┤ X ├─░─
«                          ░ └───┘ ░ 
«c0: 4/══════════════════════════════
«

In the above figure, the vertical lines define a segmentation of the state preparation and act as barriers to ensure that all gates are finished by that point. 

1. The test instance is prepared within the first barrier. A uniform superposition is formed between the ancilla and index qubits, which are registers 0 and 1, respectively. The test instance is entangled with the ground state of the ancilla. 

2. The first training instance is created between the first and second barriers. For this, the state ∣1⟩ is prepared and entangled with the excited state of the ancilla and the ground state of the index qubit with a Toffoli gate and a Pauli X gate. The Toffoli gate is also called the controlled-controlled-not gate , describing its action. 

3. The second training instance is prepared and entangled with the excited state of the ancilla and the index qubit in the third section. 

4. The class qubit gets flipped in the final part and creates the connection between the encoded training instances and the corresponding class label.